In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### read the raw data

In [2]:
data = pd.read_csv('definitions24.csv', encoding='latin1')

# create a boolean indicator =True if the definition is of "machine learning"
data['ml'] = data['term'] == 'machine learning'

print(data.dtypes)
print(data.head(10))

term          object
definition    object
ml              bool
dtype: object
               term                                         definition    ml
0  machine learning  Automatic or semi-automatic updating of equati...  True
1  machine learning  A method to train a machine(any kind) in a hum...  True
2  machine learning  Pattern recognition through a set of constantl...  True
3  machine learning  Machine learning is a predictive computational...  True
4  machine learning  Machine learning is the development of functio...  True
5  machine learning  Machine learning is the application of algorit...  True
6  machine learning  A process where algorithms, paired with data, ...  True
7  machine learning                     Algorithms that can be trained  True
8  machine learning  Machine Learning describes algorithms that can...  True
9  machine learning              Introducing an algorithm to a machine  True


### <span style="color:red">create feature set X (matrix) and vector of labels L</span>

Use [feature extraction methods in scikit-learn](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) to *vectorize* the text of machine learning/AI definitions into an X matrix. 

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

L = data["ml"] # labels
corpus = data['definition'] # corpus of reviews in words
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(corpus)
print(X.shape)

(313, 1209)


### let's look at the words in the corpus of "definition"

In [4]:
print(len(vectorizer.vocabulary_))
print("* * * * *")
print(vectorizer.vocabulary_)

1209
* * * * *
{'automatic': 94, 'or': 738, 'semi': 927, 'updating': 1134, 'of': 720, 'equations': 358, 'parameters': 763, 'using': 1143, 'algorithms': 43, 'automated': 93, 'parameter': 762, 'curve': 253, 'response': 898, 'surface': 1024, 'fitting': 416, 'method': 658, 'to': 1091, 'train': 1099, 'machine': 626, 'any': 63, 'kind': 572, 'in': 511, 'human': 491, 'manner': 640, 'achieve': 12, 'the': 1062, 'goal': 449, 'copying': 235, 'behavior': 113, 'even': 364, 'better': 125, 'than': 1059, 'regular': 879, 'being': 117, 'pattern': 769, 'recognition': 867, 'through': 1085, 'set': 937, 'constantly': 229, 'modifying': 676, 'rules': 910, 'learning': 592, 'is': 561, 'predictive': 808, 'computational': 216, 'based': 105, 'on': 727, 'training': 1101, 'model': 671, 'ml': 669, 'algorithm': 41, 'data': 258, 'followed': 419, 'by': 149, 'an': 52, 'optimization': 737, 'process': 819, 'whereby': 1177, 'tested': 1057, 'with': 1189, 'test': 1056, 'increase': 516, 'accuracy': 10, 'prediction': 806, 'devel

### <span style="color:red">fit X, L to SVM using gradient descent</span>

[gradient descent documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

In [5]:
# fit SVM linear classifier
from sklearn import linear_model
sgd = linear_model.SGDClassifier(n_iter_no_change=20)
sgd.fit(X, L)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=20, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [6]:
sgd.n_iter_

29

### assess performance

In [7]:
# look at performance measures
import my_measures

performance_measures = my_measures.BinaryClassificationPerformance(sgd.predict(X), L, 'sgd')
performance_measures.compute_measures()
print(performance_measures.performance_measures)

{'Pos': 103, 'Neg': 210, 'TP': 103, 'TN': 210, 'FP': 0, 'FN': 0, 'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'desc': 'sgd'}


![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png)

[source](https://en.wikipedia.org/wiki/Precision_and_recall)

### test model on unseen definitions

In [8]:
# machine learning definitions
# Wikipedia, Expert System, Tech Emergence
ml_defs = ["Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.",
          "Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.",
          "Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions."]

for d in ml_defs:
    print(d)
    print("* * *")

Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.
* * *
Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.
* * *
Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions.
* * *


In [9]:
# AI definitions
# Wikipedia, Oxford dictionary
ai_defs = ["Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.",
          "the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages"]

for d in ai_defs:
    print(d)
    print("* * *")

Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.
* * *
the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages
* * *


In [10]:
# Definitions of unrelated things: kitten, piano, widget
other_defs = ["A kitten, also known as a kitty or kitty cat, is a juvenile cat.",
             "The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.",
             "a small gadget or mechanical device, especially one whose name is unknown or unspecified"]

for d in other_defs:
    print(d)
    print("* * *")

A kitten, also known as a kitty or kitty cat, is a juvenile cat.
* * *
The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.
* * *
a small gadget or mechanical device, especially one whose name is unknown or unspecified
* * *


### function to transform new definitions to a X vector

In [11]:
def get_prediction(definition):
    text_x = vectorizer.transform([definition]).toarray()
    return(sgd.predict(text_x))

### view predicted classifications of new definitions

In [12]:
print("Model predictions for 'machine learning' definitions:")
for mld in ml_defs:
    print(get_prediction(mld))
    
print("* * *")
print("Model predictions for 'AI' definitions:")
for aid in ai_defs:
    print(get_prediction(aid))

print("* * *")
print("Model predictions for other definitions (kitten, piano, widget):")
for otherd in other_defs:
    print(get_prediction(otherd))

Model predictions for 'machine learning' definitions:
[False]
[ True]
[ True]
* * *
Model predictions for 'AI' definitions:
[False]
[False]
* * *
Model predictions for other definitions (kitten, piano, widget):
[False]
[False]
[False]
